In [1]:
#import re  # Для препроцессинга
import pandas as pd  # Для обработки данных
from time import time  # Для определения времени операций
import spacy
from spacy.lang.ru.examples import sentences 
from collections import defaultdict  # Для частоты слов
import logging  # Настройка для gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

In [2]:
# Загружаю данные
df_clean = pd.read_csv(r'PRE.csv')

In [3]:
df_clean.head(-5)

,clean
0,прекрасный однокомнатный квартира дом лифт мус...
1,светлый просторный комната кв метр отдельный л...
2,первомайский мина пешкомт плая уютный квартира...
3,шаговый доступность парк сокольники развитой и...
4,ильинский коммуникация центральный квартира хо...
...,...
100713,переуступка входить цена корпус комната изолир...
100714,продаваться трехкомнатная квартира замечательн...
100715,прочитайте объявление полностью звонить строго...
100716,читать звоним согласно статья фз исполнительны...


In [4]:
#использую реализацию Gensim Doc2Vec
import multiprocessing
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

In [5]:
cores = multiprocessing.cpu_count() # Количество ядер на компе

In [6]:
df_clean['clean']=df_clean['clean'].fillna("Без описания")# меняю нан на "Без описания"

In [7]:
#токенизация и добавление в текст
description = [TaggedDocument(doc.split(' '), [i]) 
             for i, doc in enumerate(df_clean.clean)]

In [8]:
#размер вектора, количество измерений, количество переборов, количество ядер
d2v_model = Doc2Vec(vector_size=64, min_count=1, epochs = 20, workers=cores)

INFO - 22:41:18: Doc2Vec lifecycle event {'params': 'Doc2Vec(dm/m,d64,n5,w5,s0.001,t16)', 'datetime': '2022-11-27T22:41:18.859189', 'gensim': '4.1.2', 'python': '3.9.13 (main, Aug 25 2022, 23:51:50) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19044-SP0', 'event': 'created'}


In [9]:
#Модель
d2v_model = Doc2Vec(vector_size=64, window=2, min_count=1, workers=cores, epochs = 30)
#Словарь
d2v_model.build_vocab(description)
#Обучение модели
d2v_model.train(description, total_examples=d2v_model.corpus_count
            , epochs=d2v_model.epochs)

INFO - 22:41:21: Doc2Vec lifecycle event {'params': 'Doc2Vec(dm/m,d64,n5,w2,s0.001,t16)', 'datetime': '2022-11-27T22:41:21.706078', 'gensim': '4.1.2', 'python': '3.9.13 (main, Aug 25 2022, 23:51:50) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19044-SP0', 'event': 'created'}
INFO - 22:41:21: collecting all words and their counts
INFO - 22:41:21: PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
INFO - 22:41:21: PROGRESS: at example #10000, processed 633411 words (7565609/s), 14572 word types, 0 tags
INFO - 22:41:21: PROGRESS: at example #20000, processed 1224343 words (6987911/s), 19483 word types, 0 tags
INFO - 22:41:21: PROGRESS: at example #30000, processed 1776316 words (7097961/s), 21530 word types, 0 tags
INFO - 22:41:22: PROGRESS: at example #40000, processed 2331132 words (6997548/s), 23159 word types, 0 tags
INFO - 22:41:22: PROGRESS: at example #50000, processed 3260837 words (7983315/s), 25968 word types, 0 tags
INFO - 22:41:22: PROGRESS: a

In [10]:
#Генерация векторов
des2vec = [d2v_model.infer_vector((df_clean['clean'][i].split(' '))) for i in range(0,len(df_clean['clean']))]
des2vec

[array([ 0.26291844, -0.04466333, -0.02650302, -0.13689099,  0.066483  ,
        -0.42220876,  0.42972612, -0.30051973,  0.09077042, -0.68868876,
        -0.28195268,  0.3318693 ,  0.13211502,  0.14467631,  0.34785068,
         0.01585725,  0.4510162 , -0.07689901,  0.5447195 , -0.5710132 ,
        -0.33899012,  0.03252968,  0.03059241,  0.13752782,  0.04420403,
         0.25442633, -0.00091957,  0.07420242, -0.02414076,  0.1479607 ,
        -0.11040697, -0.19998999, -0.21229404, -0.07435347, -0.06260704,
         0.14940834, -0.04539711, -0.250935  ,  0.01241416,  0.3303603 ,
        -0.04484077,  0.0282403 ,  0.46244505,  0.10210874, -0.2536181 ,
         0.38674968, -0.17848451, -0.51142013,  0.29005536,  0.6518787 ,
        -0.08619046,  0.26363203, -0.21366729, -0.00638447, -0.1920179 ,
         0.06265379, -0.11534686,  0.22293554, -0.23988087,  0.22628716,
         0.14221542, -0.10313563, -0.58630705,  0.03800784], dtype=float32),
 array([ 0.382445  ,  0.12732837, -0.19766262, 

In [11]:
#Проверяю тип данных
des2vec[0].dtype


dtype('float32')

In [ ]:
#Вычисляю среднее значение векторов
#from statistics import mean 
#vec=[]
#for i in range (0,len(des2vec),1):
    #vec1 = des2vec[i] 
    #vec_mean= mean(vec1)
    #vec.append(vec_mean)
    #i=i+1

#vec

In [12]:
import numpy as np
#Создание списка списков
#добавление списка в столбец данных
df_clean['doc2vec'] = np.array(des2vec).tolist()

#Это не обязательное условие
#df_clean= df_clean.drop([0],axis=1)

df_clean.head(-5)

,clean,doc2vec
0,прекрасный однокомнатный квартира дом лифт мус...,"[0.2629184424877167, -0.044663332402706146, -0..."
1,светлый просторный комната кв метр отдельный л...,"[0.38244500756263733, 0.12732836604118347, -0...."
2,первомайский мина пешкомт плая уютный квартира...,"[0.2743997871875763, 0.183375284075737, 0.2879..."
3,шаговый доступность парк сокольники развитой и...,"[0.22142431139945984, 0.618075430393219, 0.436..."
4,ильинский коммуникация центральный квартира хо...,"[0.5394482016563416, -0.4186371862888336, 0.03..."
...,...,...
100713,переуступка входить цена корпус комната изолир...,"[0.35492849349975586, 0.03739245608448982, -0...."
100714,продаваться трехкомнатная квартира замечательн...,"[0.1337912380695343, 0.4326677918434143, 0.375..."
100715,прочитайте объявление полностью звонить строго...,"[0.23989219963550568, -0.2145800143480301, 0.4..."
100716,читать звоним согласно статья фз исполнительны...,"[-0.006261778064072132, -0.5219048857688904, 0..."


In [13]:
#Сохраняю данные
df_clean.to_csv(r'2vec.csv',index=False)

In [14]:
#считываю csv для объединеня
df1 = pd.read_csv(r'Сlean.csv')
df2 = pd.read_csv(r'2vec.csv')

In [15]:
#конкатенация
Data = pd.concat([df1, df2], axis=1)
Data.head(2)

,id,rooms,cost(₽),square(м²),cost_for_meter(₽),address,metro_name,distance(м),center_distance/m,floor,floors,description,clean,doc2vec
0,0,1 – комнатная,5000000.0,31.0,161290.3,"Центральная ул., 6",бульвар дмитрия донского,4995.0,20600.0,2,4,Прекрасная однокомнатная квартира в доме с лиф...,прекрасный однокомнатный квартира дом лифт мус...,"[0.2629184424877167, -0.044663332402706146, -0..."
1,1,1 комната,4500000.0,98.0,45918.4,"Коптевская ул., 28к2",коптево,420.0,11400.0,3,5,"Светлая, просторная комната 19 кв. метров. Отд...",светлый просторный комната кв метр отдельный л...,"[0.38244500756263733, 0.12732836604118347, -0...."


In [16]:
#Удаляю столбец с предобработкой текста
Data=Data.drop(['clean'],axis=1)
Data.head(2)

,id,rooms,cost(₽),square(м²),cost_for_meter(₽),address,metro_name,distance(м),center_distance/m,floor,floors,description,doc2vec
0,0,1 – комнатная,5000000.0,31.0,161290.3,"Центральная ул., 6",бульвар дмитрия донского,4995.0,20600.0,2,4,Прекрасная однокомнатная квартира в доме с лиф...,"[0.2629184424877167, -0.044663332402706146, -0..."
1,1,1 комната,4500000.0,98.0,45918.4,"Коптевская ул., 28к2",коптево,420.0,11400.0,3,5,"Светлая, просторная комната 19 кв. метров. Отд...","[0.38244500756263733, 0.12732836604118347, -0...."


In [17]:
#Сохраняю данные
Data.to_csv(r'Data.csv',index=False)